In [1]:
#step 2:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/kafka/3.4.0/kafka_2.13-3.4.0.tgz
!tar -xzf kafka_2.13-3.4.0.tgz

--2023-05-01 22:03:06--  https://downloads.apache.org/kafka/3.4.0/kafka_2.13-3.4.0.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106290956 (101M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.4.0.tgz’

kafka_2.13-3.4.0.tg 100%[===================>] 101.37M  30.8MB/s    in 3.8s    

2023-05-01 22:03:10 (26.6 MB/s) - ‘kafka_2.13-3.4.0.tgz’ saved [106290956/106290956]



In [ ]:
import subprocess
import os
zk = subprocess.Popen(["./kafka_2.13-3.4.0/bin/zookeeper-server-start.sh", "./kafka_2.13-3.4.0/config/zookeeper.properties"])
os.system("sleep 5")
kf = subprocess.Popen(["./kafka_2.13-3.4.0/bin/kafka-server-start.sh", "./kafka_2.13-3.4.0/config/server.properties"])
os.system("sleep 5")

In [ ]:
zk.terminate()
kf.terminate()

In [ ]:
!pip install kafka-python


In [ ]:
from kafka import KafkaProducer, KafkaConsumer

In [ ]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])


In [ ]:
producer.send('your_topic', b'your_message')


In [ ]:
consumer = KafkaConsumer('your_topic', bootstrap_servers=['localhost:9092'], auto_offset_reset='latest', enable_auto_commit=True)


ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=ad2d1ed60a8a:9092 <connecting> [IPv4 ('172.28.0.12', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]> returned error 99. Disconnecting.


NoBrokersAvailable: ignored

In [ ]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

spark = SparkSession.builder.appName("DataPreprocessing").getOrCreate()
df = spark.read.csv(pathh, header=True, inferSchema=True)
pathh = "/content/drive/MyDrive/uber-raw-data-aug14(1).csv"


train_data, test_data = df.randomSplit([0.8, 0.2], seed=20)

print(f"Kafka : Sending Test_data rows using Kafka Producer. total rows: {test_data.count()}.")
for row in test_data.toJSON().collect():
    producer.send("sodfkaofasmoas", row.encode('utf-8'))

producer.close()
print("Kafka : Sending Test_data Complete.")

In [ ]:
from kafka import KafkaProducer
from pyspark.sql import SparkSession

# create Kafka producer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

# create Spark session
spark = SparkSession.builder.appName("DataPreprocessing").getOrCreate()

# read data from CSV file
pathh = "/content/drive/MyDrive/uber-raw-data-aug14(1).csv"
df = spark.read.csv(pathh, header=True, inferSchema=True)

# split data into train and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=40)

# send test data to Kafka topic
print(f"Sending {test_data.count()} rows of test data to Kafka topic.")
for row in test_data.toJSON().collect():
    producer.send("my-kafka-topic", row.encode('utf-8'))

producer.flush()
producer.close()

print("Test data sent to Kafka topic successfully.")

In [ ]:
consumer = KafkaConsumer(
    'sodfkaofasmoas',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my_group'
)
input_col_order = ["Lat","Lon","Date/Time-seconds","Base-Numerical"]

row_counter=0
def create_dataframe_and_predict(messages):
    global Final_df
    global row_counter
    string_list = []
    for x in messages[0]:
        message_str = x.value.decode('utf-8')
        string_list.append(message_str)
    print(f"Kafka : Received {len(string_list)} rows.")
    dict_list = [json.loads(s) for s in string_list]
    df = spark.createDataFrame(dict_list)
    assembler = VectorAssembler(inputCols=input_col_order, outputCol="features")
    df = assembler.transform(df)
    predictions = model.transform(df)
    cache_df=Final_df.union(predictions)
    Final_df=cache_df
    row_counter=row_counter+len(string_list)
    print(f"Model : Predicting {len(string_list)} rows complete. appended to Final_df. Total rows predicted: {row_counter}")

print("Kafka is ready to get rows.")
start_time = time.time()
unsaved=False
while True:
    messages = [msg for msg in consumer.poll(max_records=5000).values()]
    if messages:
        time.sleep(0.01)
        start_time = time.time()
        unsaved=True
        create_dataframe_and_predict(messages)
    if unsaved:
        if (time.time() - start_time)>10:
            print(f"Kafka : got nothing for 10 secs. Saving predicts in System.\nKafka : Total Predicts: {Final_df.count()} saved in path A:/Uber-Predicts.")
            time.sleep(0.01)
            Final_df.select("Lat", "Lon", "Date/Time-seconds","Base-Numerical", "prediction").coalesce(1).write.mode("overwrite").csv("A:/Uber-Predicts")
            unsaved=False